Grab relevant information for bus stops for a specific route, join the data together.

In [1]:
import requests
import pandas as pd
import glob
import json
import time
import pymssql

with open('config.json', 'r') as f:
    config = json.load(f)

SWIFTLY_API_KEY = config['DEFAULT']['SWIFTLY_API_KEY']
MSSQL_USERNAME = config['DEFAULT']['MSSQL_USERNAME']
MSSQL_PASSWORD = config['DEFAULT']['MSSQL_PASSWORD']

connection = pymssql.connect(server='ELTDBPRD\ELTDBPRD', user=MSSQL_USERNAME, password=MSSQL_PASSWORD, database='ACS_13')
# mssql_login = config['DEFAULT']['MSSQL_AMIGO_ADMIN']

# DEBUG = True

# October dates [2,3,4,5,9,10,11,12,16,17,18,19,23,24,25,26,30]

In [2]:
def pull_ridership_by_stop(line_number):

    # allFiles = glob.glob('.' + "/gps_*.csv")
    frame = pd.DataFrame()
    frames = []
    for file_ in ['Ridership/WEEKDAY.XLSX','Ridership/LRTWEEKDAY.XLSX']:
        df = pd.read_excel(file_, header=0)
        df['DAY']='RS_WKDY'
        #     df['DAY']='Weekday'
        frames.append(df)
    # for file_ in ['Ridership/SATURDAY.XLSX','Ridership/LRTSATURDAY.XLSX']:
    #     df = pd.read_excel(file_, header=0)
    #     df['DAY']='RS_SAT'
    #     frames.append(df)
    # for file_ in ['Ridership/SUNDAY.XLSX','Ridership/LRTSUNDAY.XLSX']:
    #     df = pd.read_excel(file_, header=0)
    #     df['DAY']='RS_SUN'
    #     frames.append(df)

    df = pd.concat(frames)
    df = df[~df['ROUTE_NUMBER'].isin([911,912,913,914])]

    df = df.query("DAY=='RS_WKDY'&ROUTE_NUMBER=='%d'" % line_number)

    #     pd.pivot_table(df.reset_index(), index=["STOP_ID"],values=["BOARD_ALL",'ALIGHT_ALL']).head()

    rid_line = pd.pivot_table(df.reset_index(), index=["STOP_ID","DIRECTION_NAME","TIME_PERIOD"],values=["SORT_ORDER","BOARD_ALL",'ALIGHT_ALL','LOAD_ALL','AVG_SERVICED','TIME_PERIOD_SORT']).reset_index().sort_values(by=['DIRECTION_NAME','TIME_PERIOD_SORT','SORT_ORDER'])
    rid_line['ALIGHT_ALL']= rid_line['ALIGHT_ALL'].round(2)
    rid_line['AVG_SERVICED'] = rid_line['AVG_SERVICED'].round(2)
    rid_line['BOARD_ALL'] = rid_line['BOARD_ALL'].round(2)
    rid_line['LOAD_ALL'] = rid_line['LOAD_ALL'].round(2)
    return rid_line

In [3]:
def pull_early_late_by_stop(line_number,SWIFTLY_API_KEY, dateRange, timeRange):
    '''http://dashboard.goswift.ly/vta/api-guide/docs/otp'''
#     line_number = '22'
#     dateRange = '10012017-10302017'
#     timeRange = '0500-0900'
    line_table = pd.read_csv('line_table.csv')
    line_table.rename(columns={"DirNum":"direction_id","DirectionName":"DIRECTION_NAME"},inplace=True)
    line_table['direction_id'] = line_table['direction_id'].astype(str)
    headers = {'Authorization': SWIFTLY_API_KEY}
    payload = {'agency': 'vta', 'route': line_number, 'dateRange': dateRange,'timeRange': timeRange, 'onlyScheduleAdherenceStops':'True'}
    # payload = {'agency': 'vta', 'dateRange': '10292017-10302017'}
    url = 'https://api.goswift.ly/otp/by-stop'
    r = requests.get(url, headers=headers, params=payload)
    print(r.text)
    try:
        swiftly_df = pd.DataFrame(r.json()['data'])
        swiftly_df.rename(columns={"stop_id":"STOP_ID"},inplace=True)
        swiftly_df = pd.merge(swiftly_df,line_table.query('lineabbr==%s'%line_number)[['direction_id','DIRECTION_NAME']])
        swiftly_df['STOP_ID'] = swiftly_df['STOP_ID'].astype(int)
        return swiftly_df
    except KeyError:
        print(r.json())

In [4]:
def stop_frequency_percent():
    sql = '''select current_route_id, k.direction_code_id, dc.[direction_description], sum(count_trips) as unique_trips_sampled from (
SELECT [direction_code_id], current_route_id
      ,count(distinct(ext_trip_id)) as 'count_trips'
      ,datepart(dy, [apc_date_time]) as 'dayofyear'
      /*,cast(floor(cast([apc_date_time] as float)) as datetime) as indate*/
      FROM [ACS_13].[dbo].[apc_correlated] where 
  (apc_date_time between '2017-10-03' and '2017-10-06' or 
  apc_date_time between '2017-10-10' and '2017-10-13' or 
  apc_date_time between '2017-10-17' and '2017-10-20' or
  apc_date_time between '2017-10-24' and '2017-10-27' or
  apc_date_time between '2017-10-31' and '2017-11-1') and 
  current_route_id = 22 
  and bs_id != 0
  group by current_route_id, direction_code_id ,datepart(dy, [apc_date_time])
  ) k 
  LEFT join [ACS_13].[dbo].[direction_codes] dc
  on k.direction_code_id = dc.[direction_code_id]

  group by current_route_id, k.direction_code_id, dc.[direction_description]'''

    trips_sampled = pd.read_sql(sql,connection)
    
    sql = '''SELECT [direction_code_id]
      , [bs_id],
      count(bs_id) as 'number of times stopped'
      FROM [ACS_13].[dbo].[apc_correlated] where 
  (apc_date_time between '2017-10-03' and '2017-10-06' or 
  apc_date_time between '2017-10-10' and '2017-10-13' or 
  apc_date_time between '2017-10-17' and '2017-10-20' or
  apc_date_time between '2017-10-24' and '2017-10-27' or
  apc_date_time between '2017-10-31' and '2017-11-1') and 
  current_route_id = 22 
  and bs_id != 0
  group by direction_code_id, bs_id
  order by direction_code_id, bs_id'''

    number_of_times_stopped_in_period = pd.read_sql(sql,connection)
    stopped_frequency = pd.merge(number_of_times_stopped_in_period, trips_sampled)
    
    stopped_frequency['% frequency stopped'] = (stopped_frequency['number of times stopped']/stopped_frequency['unique_trips_sampled']).round(2)
    stopped_frequency.rename(columns={"bs_id":"STOP_ID","direction_description":"DIRECTION_NAME"},inplace=True)

    return stopped_frequency

In [5]:
def dwell_runtime(line_number):
    line_number = 22
    allFiles = glob.glob('.' + "/swiftly data oct/*.csv")
    frame = pd.DataFrame()
    frames = []
    # check for Too many lines errors
    # count = 0
    # for file_ in allFiles:
    #     count = count + 1
    #     print(file_, read_first_lines(file_, 1))
    for file_ in allFiles:
        df = pd.read_csv(file_)
        frames.append(df)
    df = pd.concat(frames, ignore_index=True)
    df['time'] = pd.to_datetime(df['actual_date'] + ' ' + df['actual_time'], format='%m-%d-%y %H:%M:%S')

    # 1430-1829 = pm peak
    def TIME_PERIOD(x):
    #     if df['time'].dt.hour >= 5 and df['time'].dt.hour <= 8:
        if x.hour >= 5 and x.hour <= 8:
            return 'AM Peak'
        elif x.hour >= 14 and x.hour <= 18:
            return 'PM Peak'
        else:
            return 'Neither time zone'

    df = df.query("route_id=='%d'" % line_number)
    df['TIME_PERIOD'] = df['time'].apply(TIME_PERIOD)

    f = {'dwell_time_secs':['mean','std','size']}
    # .size()
    df_dwell = df.query("route_id=='%d'&is_departure==True" % line_number).groupby(['route_id','direction_id','TIME_PERIOD','stop_id']).agg(f)

    f = {'travel_time_secs':['mean','std','size']}
    df_runtime = df.query("route_id=='%d'&is_departure==False" % line_number).groupby(['route_id','direction_id','TIME_PERIOD','stop_id']).agg(f)

    df_results = pd.merge(df_dwell.reset_index(), df_runtime.reset_index())
    # , 'B':['prod']

    line_table = pd.read_csv('line_table.csv')
    line_table.rename(columns={"DirNum":"direction_id","DirectionName":"DIRECTION_NAME", "lineabbr":"route_id"},inplace=True)
    line_table['direction_id'] = line_table['direction_id'].astype(int)
    # line_table.direction_id = line_table.direction_id.astype(int)
    df_results = pd.merge(df_dwell.reset_index(), df_runtime.reset_index())
    df_results.rename(columns={'dwell_time_secs':'dwell_time_secs_','travel_time_secs':'travel_time_secs_'},inplace=True)
    df_results.columns = [''.join(t) for t in df_results.columns]
    df_results = pd.merge(df_results,df.groupby(['route_id','direction_id','stop_id'])['stop_path_length_meters'].max().reset_index(),how='left')
    df_results.rename(columns={'stop_id':'STOP_ID'},inplace=True)
    return pd.merge(df_results,line_table, how='left', left_on = ['route_id','direction_id'], right_on = ['route_id','direction_id'])

In [ ]:
rid_by_stop_df = pull_ridership_by_stop(22)
rid_by_stop_df.head()

,STOP_ID,DIRECTION_NAME,TIME_PERIOD,ALIGHT_ALL,AVG_SERVICED,BOARD_ALL,LOAD_ALL,SORT_ORDER,TIME_PERIOD_SORT
186,328,EAST,AM Early,0.49,0.99,43.87,43.99,10,1
192,329,EAST,AM Early,0.00,0.19,0.64,44.62,20,1
198,330,EAST,AM Early,0.00,0.05,0.16,44.79,30,1
204,331,EAST,AM Early,0.00,0.01,0.04,44.83,40,1
210,332,EAST,AM Early,0.00,0.00,0.00,44.83,50,1


In [ ]:
df_dwell_runtime = dwell_runtime(22)

In [ ]:
rid_dwell = pd.merge(rid_by_stop_df,df_dwell_runtime,how='left')

In [ ]:
# Midday - 9 to 2:30
# PM Peak - 2:30 to 6:30
# PM Late - 6:30 to 9:59
# PM Nite - 10pm to 12pm
# PM Nite - 12am to 3am

In [ ]:
frames = []

# df = pull_early_late_by_stop('22',SWIFTLY_API_KEY, dateRange = '10012017-10302017', timeRange = '0300-0459')
# df['TIME_PERIOD'] = 'AM Early'
# frames.append(df)
# time.sleep(10)
df = pull_early_late_by_stop('22',SWIFTLY_API_KEY, dateRange = '10012017-10302017', timeRange = '0500-0829')
df['TIME_PERIOD'] = 'AM Peak'
frames.append(df)
time.sleep(10)
# df = pull_early_late_by_stop('22',SWIFTLY_API_KEY, dateRange = '10012017-10302017', timeRange = '0900-1429')
# df['TIME_PERIOD'] = 'Midday'
# frames.append(df)
# time.sleep(10)
df = pull_early_late_by_stop('22',SWIFTLY_API_KEY, dateRange = '10012017-10302017', timeRange = '1430-1829')
df['TIME_PERIOD'] = 'PM Peak'
frames.append(df)
# time.sleep(10)
# df = pull_early_late_by_stop('22',SWIFTLY_API_KEY, dateRange = '10012017-10302017', timeRange = '1830-2199')
# df['TIME_PERIOD'] = 'PM Late'
# frames.append(df)
# time.sleep(10)
# df = pull_early_late_by_stop('22',SWIFTLY_API_KEY, dateRange = '10012017-10302017', timeRange = '2200-2359')
# df['TIME_PERIOD'] = 'PM Nite'
# frames.append(df)
# time.sleep(10)
# df = pull_early_late_by_stop('22',SWIFTLY_API_KEY, dateRange = '10012017-10302017', timeRange = '0000-0259')
# df['TIME_PERIOD'] = 'PM Nite'
# frames.append(df)

In [ ]:
times = pd.concat(frames)
times['TIMEPOINT'] = True

In [ ]:
times.head()

In [ ]:
bus_df = pd.merge(rid_dwell,times, how='outer', on=['STOP_ID','DIRECTION_NAME','TIME_PERIOD'])
# bus_df.to_csv("bus_analysis.csv",index=False)
# bus_df.head()

In [ ]:
stop_frequency = stop_frequency_percent()

In [ ]:
# bus_df
bus_df_frequency = pd.merge(rid_dwell, stop_frequency)

In [ ]:
bus_df_frequency.to_csv("bus_analysis.csv",index=False)

In [ ]:
bus_df_frequency.query('TIMEPOINT==True')